Tutorials:
- https://rasterio.readthedocs.io/en/stable/topics/resampling.html
- https://pygis.io/docs/e_raster_resample.html

In [1]:
from ipyleaflet import Choropleth, Map, basemap_to_tiles, LayersControl
import ipywidgets as widgets
from IPython.display import display
from localtileserver import get_leaflet_tile_layer, TileClient
import matplotlib
# from Magics import macro as magics
# from Magics.macro import *
import numpy as np
import pandas as pd
import rasterio #rasterio was preferred over xarray since it is more compatible with ipyleaflet
from rasterio.enums import Resampling
import requests
import json
# import rioxarray as rxr
# import xarray as xr

In [2]:
#resample function
def resample_raster(path, fact = 0.5, rio = True, nodata = -3.40282e+38):
    """
    Resample a raster from its file path applying a factor
    Issues:
        - This resampling method will create artifacts in areas with nodata
        - Does not work with Resampling.sum, which was the algorithm needed to 
    
    path: str
        Path to raster file
    fact: float
        Upscaling or downscaling factor. Default: 0.5 (i.e. halving the raster spatial resolution)
    """
    
    outpath = path.split(".")[0] + "_resampled_" + str(fact) + "." + path.split(".")[1]
    if rio:
        with rasterio.open(path, "r") as dataset:
            # mask no data


            # resample data to target shape
            data = dataset.read(
                out_shape=(
                    dataset.count,
                    int(dataset.height * fact),
                    int(dataset.width * fact)
                ),
                resampling = Resampling.bilinear,
                masked = True
            )
            # scale image transform
            transform = dataset.transform * dataset.transform.scale(
                (dataset.width / data.shape[-1]),
                (dataset.height / data.shape[-2])
            )

            # Write outputs
            # set properties for output
            dst_kwargs = dataset.meta.copy()
            dst_kwargs.update(
                {
                    "crs": dataset.crs,
                    "transform": transform,
                    "width": data.shape[-1],
                    "height": data.shape[-2],
                    "nodata": 0,  
                }
            )
            with rasterio.open(outpath, "w", **dst_kwargs) as dst:
                # iterate through bands
                for i in range(data.shape[0]):
                        dst.write(data[i].astype(rasterio.float64), i+1)
        return(dataset)
    else:
        # method with xarray if needed
        # https://docs.xarray.dev/en/stable/generated/xarray.DataArray.coarsen.html
        pass

## Resample global pop raster

Starting resolution: 1 km (0.0083 degrees) \
Goal resolution: 10 km (0.083 degrees) \
Scaling factor: 0.1

In [16]:
# CAREFUL:
# this produces a bilinear resampled raster, which is not what it is needed for population data
# we need to sum the population in each cell, but this can not be done in this way and I haven't found a way to do it through rasterio
#I've done it through QGIS and uploaded the result in data/impacts folder

path = "C:/Users/paolo/OneDrive - Politecnico di Milano/code4earth/ppp_2020_1km_Aggregated.tif"
resampled = resample_raster(path, 0.1)

In [ ]:
## Resample single countries rasters